In [1]:
import streamlit as st

In [2]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import seaborn as sns
from keras.models import load_model
import pandas_datareader as data
from datetime import date
import joblib
from sklearn.preprocessing import MinMaxScaler

In [ ]:
START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

In [ ]:
st.title('Stock Prediction using LSTM')

In [ ]:
stocks = ("NVDA","BCS","PFE")
selected_stock = st.selectbox("Select dataset for prediction", stocks)

In [ ]:
@st.cache_data

In [ ]:
def load_data(ticker):
    data = yf.download(ticker, START,TODAY)
    return data

In [ ]:
data_load_state = st.text("Load data. . .")
data = load_data(selected_stock)
data_load_state.text("Loading data . . . done!")

In [ ]:
st.subheader('Data from 2015')
st.write(data.describe())

In [ ]:
def plot_raw_data():
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=data.index , y= data['Adj Close'], name = 'Stock_AdjClose'))
    fig1.add_trace(go.Scatter(x=data.index , y= data['Open'], name = 'Stock_Open'))
    fig1.layout.update(title_text='Time Series Data', xaxis_rangeslider_visible=True)
    st.plotly_chart(fig1)

In [ ]:
plot_raw_data()

In [ ]:
def fetch_data():
    # Fetch or load your data here
    # For example, you might use Pandas to read from a CSV file or fetch from a database
    return data

In [ ]:
st.subheader('Closing Price vs Time chart with 100MA & 200MA')
ma100 = data.Close.rolling(100).mean()
ma200 = data.Close.rolling(200).mean()
fig3 = plt.figure(figsize = (12,6))
plt.plot(ma100, 'red', label='100MA')
plt.plot(ma200, 'blue',label='200MA')
plt.plot(data.Close, 'green')
plt.legend()
st.pyplot(fig3)


In [ ]:
#jupyter nbconvert --to script Streamlit6.ipynb
#streamlit run Streamlit6.py
#%matplotlib inline

In [ ]:
features = ['Open', 'High', 'Low', 'Close']

In [ ]:
#Scaling
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(data[features])
feature_transform= pd.DataFrame(columns=features, data=feature_transform,index=data.index)

In [ ]:
df1 = data[['Adj Close']].copy()

In [ ]:
# Scaling target data
scaler = MinMaxScaler()
df1 ['Adj Close'] = scaler.fit_transform(df1 [['Adj Close']])

In [ ]:
#Process the data for LSTM
trainX =np.array(feature_transform)

In [ ]:
X_train = trainX.reshape(feature_transform.shape[0], 1, feature_transform.shape[1])

In [ ]:
lstm = load_model(r"C:\Users\ross0\OneDrive\lstm_model.keras")

In [ ]:
#LSTM Prediction
y_pred= lstm.predict(X_train)

In [ ]:
y_pred = scaler.inverse_transform(y_pred)

In [ ]:
# Inverse transforming the scaled data
df1[['Adj Close']] = scaler.inverse_transform(df1[['Adj Close']])

In [ ]:
# Reassigning the scaled data to the 'Adj Close' column
df1 = pd.DataFrame(data=df1['Adj Close'], index=df1.index, columns=['Adj Close'])

In [ ]:
df1 = df1.assign(y_pred = y_pred)

In [ ]:
st.subheader('LSTM model vs Actual "Adj Close" price')

fig4 = plt.figure(figsize = (12,6))
plt.plot(df1['Adj Close'], 'red',label='Actual Adj Close')
plt.plot(df1['y_pred'], 'blue',label='Predicted Adj Close')
# # Setting title and labels
plt.title("Prediction by LSTM")
plt.xlabel('Date')
plt.ylabel('Adj Close Prices')

# Adding legend
plt.legend()
st.pyplot(fig4)

In [ ]:
def predict(Open, High, Low, Close):
    dx = pd.DataFrame({
        'Open': [Open],
        'High': [High],
        'Low': [Low],
        'Close': [Close]
    })
    # Calculate the average of Open, High, Low, and Close
    dx1 = dx[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    #Create a new DataFrame with the average values
    dx1 = pd.DataFrame(dx1, columns=['Average'])
    # List of features
    features = ['Open', 'High', 'Low', 'Close']
    # Scale the features
    scaler = MinMaxScaler()
    dx_feature_transform = scaler.fit_transform(dx[features])
    # Convert to DataFrame
    dx_feature_transform = pd.DataFrame(columns=features, data=dx_feature_transform)
    trainx =np.array(dx_feature_transform)
    x_train = trainx.reshape(dx_feature_transform.shape[0], 1, dx_feature_transform.shape[1])
    x_pred= lstm.predict(x_train)
    scaler = MinMaxScaler()
    dx1_feature_transform = scaler.fit_transform(dx1['Average'].values.reshape(-1, 1))
    # Convert scaled values back to DataFrame
    dx_1feature_transform = pd.DataFrame(dx1_feature_transform, columns=['Scaled_Average'])
    x_pred = scaler.inverse_transform(x_pred)
    return x_pred
    

In [ ]:
def main():
    st.title('Predict Adj Close Price for the next day')

    st.write('Enter prices in the format: Open High Low Close')
    user_input = st.text_input('Prices', '492.44 492.95 475.95 481.68')
    
    if st.button('Predict'):
        # Split the user input and convert to floats
        try:
            Open, High, Low, Close = map(float, user_input.split())
            # Call your predict function with user input
            predicted_value = predict(Open, High, Low, Close)
            st.write(f"The predicted 'Adj Close' tomorrow is {predicted_value}")
            # Display animated GIF after prediction
            st.markdown(
                """
                <style>
                @keyframes fadeIn {
                    from {
                        opacity: 0;
                    }
                    to {
                        opacity: 1;
                    }
                }
                .fadeInImage {
                    animation: fadeIn 3.5s; /* Change the duration to 3 seconds */
                }
                </style>
                """
            , unsafe_allow_html=True)
            st.image('https://i.gifer.com/CeaB.gif', use_column_width=True)
        except ValueError:
            st.error('Please enter valid prices in the format: Open High Low Close')

if __name__ == "__main__":
    main()